In [1]:
# ЧЕРНОВОЙ ВАРИАНТ INPAINT STABLE DIFFUSION
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# СОЗДАЕМ ПАПКУ PROJECT на ГУГЛ ДИСКЕ
!mkdir -p /content/drive/MyDrive/PROJECT

In [3]:
# ОТКРЫВАЕМ ССЫЛКУ и в МОЙ ДИСК - PROJECT добавляем через меню правой кнопки Систематизировать data - https://drive.google.com/drive/folders/1RKSFIntGnprref8XhFInNLXuzVA_xfJl?usp=sharing
!find /content/drive/MyDrive/PROJECT/data

/content/drive/MyDrive/PROJECT/data
/content/drive/MyDrive/PROJECT/data/images
/content/drive/MyDrive/PROJECT/data/images/56.JPG
/content/drive/MyDrive/PROJECT/data/images/56.png


In [4]:
!pip install torch==2.2.0 Pillow requests transformers diffusers accelerate gradio==3.40.0 jupyter # https://github.com/huggingface/transformers/issues/11262

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 78.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-many

In [13]:
# --extra-index-url https://download.pytorch.org/whl/cu121
!pip install numpy==1.24.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 58.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.24.2 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.2.0 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.2.0 which is incompatible.


In [14]:
from PIL import Image
import requests
from transformers import SamModel, SamProcessor
from diffusers import DiffusionPipeline, AutoPipelineForText2Image, AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

import torch
import numpy as np

In [15]:
# SAM
# Complete the following cell by loading the pretrained SAM from Facebook/Meta. Remember to:

# Move the model to the GPU by adding .to("cuda")
# Add the option torch_dtype=torch.float16 to your call of AutoPipelineForInpainting.from_pretrained
# This cell might take a couple of minutes to load.

# Load the SAM model as we have seen in the class
# Remeber to load it on the GPU by adding .to("cuda")
# at the end
model = SamModel.from_pretrained("facebook/sam-vit-base").to("cuda")

# Load the SamProcessor using the facebook/sam-vit-base
# checkpoint
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

RuntimeError: Failed to import transformers.models.sam.image_processing_sam because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
# Get the mask
# Now that you have loaded SAM, complete the following function that uses SAM to produce a segmentation mask:

def mask_to_rgb(mask):
    """
    Transforms a binary mask into an RGBA image for visualization
    """

    bg_transparent = np.zeros(mask.shape + (4, ), dtype=np.uint8)

    # Color the area we will replace in green
    # (this vector is [Red, Green, Blue, Alpha])
    bg_transparent[mask == 1] = [0, 255, 0, 127]

    return bg_transparent


def get_processed_inputs(image, input_points):

    # Use the processor to generate the right inputs
    # for SAM
    # Use "image" as your image
    # Use 'input_points' as your input_points,
    # and remember to use the option return_tensors='pt'
    # Also, remember to add .to("cuda") at the end
    inputs = processor(images=image,
                       input_points=input_points,
                       return_tensors="pt").to("cuda") # YOUR CODE HERE

    # Call SAM
    outputs = model(**inputs)

    # Now let's post process the outputs of SAM to obtain the masks
    masks = processor.image_processor.post_process_masks(
       outputs.pred_masks.cpu(),
       inputs["original_sizes"].cpu(),
       inputs["reshaped_input_sizes"].cpu()
    )

    # Here we select the mask with the highest score
    # as the mask we will use. You can experiment with also
    # other selection criteria, for example the largest mask
    # instead of the most confident mask
    best_mask = masks[0][0][outputs.iou_scores.argmax()]

    # NOTE: we invert the mask by using the ~ operator because
    # so that the subject pixels will have a value of 0 and the
    # background pixels a value of 1. This will make it more convenient
    # to infill the background
    return ~best_mask.cpu().numpy()

In [ ]:
# Now let's test what we have done so far. By executing following cell you should get a visualization of the car and it's mask.

# Let's see what happens in this cell:

# We open the image of the car and we resize it to 512 by 512 pixels (a square image). This makes things simpler for this project
# We define a few points on the image that indicate where the car is
# We use the function we have defined to generate a mask using SAM
# We visualize the mask

raw_image = Image.open("/content/drive/MyDrive/PROJECT/data/images/56.png").convert("RGB").resize((512, 512))

# These are the coordinates of two points on the car
input_points = [[[150, 170], [300, 250]]]

mask = get_processed_inputs(raw_image, input_points)

fig = make_image_grid([raw_image.resize((256, 256)), Image.fromarray(mask_to_rgb(mask)).resize((256, 256))], rows=1, cols=2)
fig

In [ ]:
# Inpainting
# Now that we have completed the SAM setup, let's move to the inpainting setup.

# Let's start by loading our inpainting pipeline. We will use the diffusers/stable-diffusion-xl-1.0-inpainting-0.1 pretrained model and the AutoPipelineForInpainting as we have seen in our diffusers demo in Lesson 5.

# Complete the following code and run it (it might take a few minutes to run):

# NOTE: you will probably see a warning similar to The config attributes {'decay'.... Please ignore it. It is a warning generated by the diffusers library that does not constitute a problem for our application

# Load the AutoPipelineForInpainting pipeline
# (remember the diffusers demo in lesson 5)
# The checkpoint we want to use is
# "diffusers/stable-diffusion-xl-1.0-inpainting-0.1"
# Remember to add torch_dtype=torch.float16 as an option

pipeline = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1",
                                                     torch_dtype=torch.float16,
                                                     variant="fp16").to("cuda")

# This will make it more efficient on our hardware
pipeline.enable_model_cpu_offload()

In [ ]:
# Now complete the following function that gets in input:

# The raw image
# The mask generated by SAM (a numpy array)
# The text prompt for the infill
# An optional negative prompt
# An optional seed for repeatibility
# The Classifier-Free Guidance Scale (CFGS). If you don't remember what this is, refer to the Text Conditioning explanation in Lesson 5

def inpaint(raw_image, input_mask, prompt, negative_prompt=None, seed=74294536, cfgs=7):

    mask_image = Image.fromarray(input_mask)

    rand_gen = torch.manual_seed(seed)

    # Use the pipeline we have created in the previous cell
    # Use "prompt" as prompt,
    # "negative_prompt" as the negative prompt,
    # raw_image as the image,
    # mask_image as the mask_image,
    # rand_gen as the generator and
    # cfgs as the guidance_scale

    image = pipeline(image=raw_image,
                     mask_image=mask_image,
                     prompt=prompt,
                     negative_prompt=negative_prompt,
                     guidance_scale=cfgs,
                     generator=rand_gen,
                     num_inference_steps=50,
                     ).images[0]

    return image

In [1]:
# Let's test our inpainting on the mask we have obtained earlier with SAM:

prompt = "Wooden chess squares measuring one meter by one meter in red and blue colors with small navigated west five corners green star in center of each" #on Mars. Studio lights, 1970s"
negative_prompt = "artifacts, low quality, distortion"

image = inpaint(raw_image, mask, prompt, negative_prompt)

In [ ]:
fig = make_image_grid([raw_image, Image.fromarray(mask_to_rgb(mask)), image.resize((512, 512))], rows=1, cols=3)
fig


In [ ]:
# Interactive app
# To make things a bit more fun, we have prepared an interactive app for you that uses the code you have completed and allow you to upload an image, run SAM, and generate the new background through a text prompt.

# Simply execute the following cell. The output will contain a preview of the app: DO NOT USE IT. Instead, you will also see a link similar to this:

# image.png

# Click on the second link (the public URL), from there you will be able to use the app much more comfortably.

# NOTE: if for any reason you need to stop the app, click on the stop icon of the jupyter interface: image-3.png then execute the next cell containing the code my_app.close

import app

my_app = app.generate_app(get_processed_inputs, inpaint)

my_app.close()